# 버블 차트 시각화

In [ ]:
# 라이브러리 모음
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import plotly.graph_objects as go
import plotly.express as px

# 모듈 모음
import modules as mds

In [ ]:
# Windows에서 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # '맑은 고딕'이 설치되어 있을 경우
plt.rcParams['axes.unicode_minus'] = False     # 마이너스(-) 부호 깨짐 방지

In [ ]:
# macOS에서 한글 폰트 설정
plt.rcParams['font.family'] = 'AppleGothic'   # macOS 기본 한글 폰트
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# 이용률 / 일사량 버블 차트 시각화

# 버블 차트용 데이터
df_all = pd.read_excel("dataset/최종일사량합친데이터.xlsx")
df_2024 = df_all[df_all["연도"] == 2024].copy()

# 숫자 열 견고화(문자/공백/'-' 처리 → 숫자)
for col in ["설비용량(MW)", "일사량(MJ/m2)", "설비이용률(%)"]:
    df_2024[col] = (
        df_2024[col].astype(str).str.replace(",", "", regex=False).str.strip()
        .replace({"": np.nan, "-": np.nan})
    )
    df_2024[col] = pd.to_numeric(df_2024[col], errors="coerce")

# 버블 크기: 최소 50 보장
df_2024["설비용량"] = df_2024["설비용량(MW)"].fillna(0).apply(lambda v: max(50, float(v)))

# 축 범위/평균선 계산
x = pd.to_numeric(df_2024["일사량(MJ/m2)"], errors="coerce")
y = pd.to_numeric(df_2024["설비이용률(%)"], errors="coerce")
xmin, xmax = float(x.min()), float(x.max())
ymin, ymax = float(y.min()), float(y.max())
padx, pady = (xmax - xmin) * 0.05, (ymax - ymin) * 0.05
mean_x, mean_y = x.mean(), y.mean()

# 지역 색상 맵(디자인 고정)
color_map = {
    "서울": "#E74C3C","부산": "#3498DB","대구": "#2ECC71","인천": "#F39C12","광주": "#9B59B6",
    "대전": "#1ABC9C","울산": "#E67E22","경기": "#34495E","강원": "#F1C40F","경북": "#E91E63",
    "경남": "#00BCD4","전북": "#FF9800","전남": "#795548","충북": "#607D8B","충남": "#4CAF50","제주": "#673AB7"
}

# 버블 차트 생성
fig = px.scatter(
    df_2024,
    x="일사량(MJ/m2)", y="설비이용률(%)",
    size="설비용량", color="지역", hover_name="지역",
    size_max=60, title="2024년 지역별 일사량과 설비이용률 비교 분석",
    color_discrete_map=color_map
)
fig.update_traces(text=None)

# 라벨 전략: 큰 버블(설비용량≥THRESH)은 고정 라벨, 작은 버블은 화살표 라벨
THRESH = 500
for _, row in df_2024.iterrows():
    if row["설비용량"] >= THRESH:
        fig.add_annotation(
            x=row["일사량(MJ/m2)"], y=row["설비이용률(%)"],
            text=f"<b>{row['지역']}</b>", showarrow=False,
            font=dict(color="white", size=11)
        )
    else:
        fig.add_annotation(
            x=row["일사량(MJ/m2)"], y=row["설비이용률(%)"],
            text=row["지역"], showarrow=True, arrowhead=2,
            ax=15, ay=-15, font=dict(color="black", size=11),
            bgcolor="rgba(255,255,255,0.6)"
        )

# 평균선(사분면) 추가 + 레이아웃
fig.update_layout(autosize=True, height=720, template="plotly_white")
fig.add_shape(type="line", x0=mean_x, y0=ymin, x1=mean_x, y1=ymax, line=dict(color="red", dash="dash", width=2))
fig.add_shape(type="line", x0=xmin, y0=mean_y, x1=xmax, y1=mean_y, line=dict(color="red", dash="dash", width=2))
fig.add_annotation(x=xmax, y=ymax, text="성과 우수 (자원↑, 효율↑)", showarrow=False, font=dict(color="green"), xanchor="right", yanchor="bottom")
fig.update_xaxes(range=[xmin - padx, xmax + padx], title="일사량 (MJ/m²)")
fig.update_yaxes(range=[ymin - pady, ymax + pady + 0.2], title="설비이용률 (%)")

# 크기 설정
fig.update_layout(width=1440,height=1080, font=dict(size=20))

fig.show()